# **MIP3-Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Mathematical Optimization Model:

Parameters:
- N = number of powerplants (in this case, 6)
- T = number of time periods (in this case, 15)
- satart_i = startup cost for powerplant i (for i = 1 to 6)
- sdown_i = shutdown cost for powerplant i (for i = 1 to 6)
- fcost_i = fixed cost of operation for powerplant i (for i = 1 to 6)
- vcost_it = variable cost for powerplant i at time t (for i = 1 to 6, t = 1 to 15)
- lb_it = lower bound of the output power for powerplant i at time t (for i = 1 to 6, t = 1 to 15)
- ub_it = upper bound of the output power for powerplant i at time t (for i = 1 to 6, t = 1 to 15)
- deltaUp_i = maximum power increase for powerplant i (for i = 1 to 6)
- deltaDown_i = maximum power decrease for powerplant i (for i = 1 to 6)
- D_t = total power demand at time t (for t=1 to 15)

Decision Variables:
- X_it = output power of powerplant i at time t (for i = 1 to 6, t = 1 to 15)
- Start_it = binary variable to represent whether powerplant i starts at time t (for i = 1 t

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the sample Python Pyomo code for the above mentioned optimization problem. I've kept the data general, but added comments specifying it as sample data.

Code:

```python
from pyomo.environ import *

# Pyomo model
model = ConcreteModel()

# Parameters
N = 6        # Number of powerplants
T = 15       # Number of time periods
# Note: All sample data mentioned below is assumed for the purpose of this code
satart = [100, 200, 150, 180, 230, 120]   # Sample startup cost data for each powerplant
sdown = [50, 60, 55, 65, 70, 45]         # Sample shutdown cost data for each powerplant
fcost = [20, 25, 22, 26, 28, 23]         # Sample fixed cost data for each powerplant
vcost = [[10]*T for _ in range(N)]        # Variable cost data (assuming same for each powerplant for all time periods)
lb = [[1]*T for _ in range(N)]            # Lower bound of output power (assuming same for each powerplant for all time periods)
ub = [[10]*T for _ in range(N)]           # Upper bound of output power (assuming same for each powerplant for all time periods)
deltaUp = [5, 6, 5, 6, 7, 5]              # Sample data for maximum power increase for each powerplant
deltaDown = [2, 3, 2, 3, 2, 3]            # Sample data for maximum power decrease for each powerplant
D = [60]*T  # Sample total power demand for each time

# Sets
model.N = RangeSet(N)
model.T = RangeSet(T)

# Variables
model.X = Var(model.N, model.T, within=NonNegativeReals)
model.Start = Var(model.N, model.T, within=Binary)
model.Shut = Var(model.N, model.T, within=Binary)
model.Run = Var(model.N, model.T, within=Binary)

# Objective
model.obj = Objective(expr=sum(satart[i]*model.Start[i+1,t] + sdown[i]*model.Shut[i+1,t] + fcost[i]*model.Run[i+1,t] + 
                           vcost[i][t-1]*model.X[i+1, t] for i in range(N) for t in model.T), sense=minimize)

# Constraints
model.Capacity_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        model.Capacity_Constraints.add(inequality(lb[i][t-1], model.X[i+1, t], ub[i][t-1]))

model.RampUp_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampUp_Constraints.add(model.X[i+1, t] <= deltaUp[i])
        else:
            model.RampUp_Constraints.add(model.X[i+1, t] - model.X[i+1, t-1] <= deltaUp[i])

model.RampDown_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampDown_Constraints.add(0 <= deltaDown[i])
        else:
            model.RampDown_Constraints.add(model.X[i+1, t-1] - model.X[i+1, t] <= deltaDown[i])

model.Power_Balance_Constraints = ConstraintList()
for t in model.T:
    model.Power_Balance_Constraints.add(sum(model.X[i+1, t] for i in range(N)) >= 1.1 * D[t-1])

model.Run_Constraint = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t], 1))
        else:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t] - model.Run[i+1,t-1] + model.Shut[i+1,t], 1))
```

This code assumes that there are no powerplants running at the start (t=1). If there are already running powerplants, model.Run and model.Start at t=1 might be set to 1. Also, this code does not consider shut down at t=1. If powerplants are assumed to be running from the start and can be shut down at t=1, we can also add model.Shut for t=1.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Pyomo model
model = ConcreteModel()

# Parameters
N = 6        # Number of powerplants
T = 15       # Number of time periods
# Note: All sample data mentioned below is assumed for the purpose of this code
satart = [100, 200, 150, 180, 230, 120]   # Sample startup cost data for each powerplant
sdown = [50, 60, 55, 65, 70, 45]         # Sample shutdown cost data for each powerplant
fcost = [20, 25, 22, 26, 28, 23]         # Sample fixed cost data for each powerplant
vcost = [[10]*T for _ in range(N)]        # Variable cost data (assuming same for each powerplant for all time periods)
lb = [[1]*T for _ in range(N)]            # Lower bound of output power (assuming same for each powerplant for all time periods)
ub = [[10]*T for _ in range(N)]           # Upper bound of output power (assuming same for each powerplant for all time periods)
deltaUp = [5, 6, 5, 6, 7, 5]              # Sample data for maximum power increase for each powerplant
deltaDown = [2, 3, 2, 3, 2, 3]            # Sample data for maximum power decrease for each powerplant
D = [60]*T  # Sample total power demand for each time

# Sets
model.N = RangeSet(N)
model.T = RangeSet(T)

# Variables
model.X = Var(model.N, model.T, within=NonNegativeReals)
model.Start = Var(model.N, model.T, within=Binary)
model.Shut = Var(model.N, model.T, within=Binary)
model.Run = Var(model.N, model.T, within=Binary)

# Objective
model.obj = Objective(expr=sum(satart[i]*model.Start[i+1,t] + sdown[i]*model.Shut[i+1,t] + fcost[i]*model.Run[i+1,t] +
                           vcost[i][t-1]*model.X[i+1, t] for i in range(N) for t in model.T), sense=minimize)

# Constraints
model.Capacity_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        model.Capacity_Constraints.add(inequality(lb[i][t-1], model.X[i+1, t], ub[i][t-1]))

model.RampUp_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampUp_Constraints.add(model.X[i+1, t] <= deltaUp[i])
        else:
            model.RampUp_Constraints.add(model.X[i+1, t] - model.X[i+1, t-1] <= deltaUp[i])

model.RampDown_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampDown_Constraints.add(0 <= deltaDown[i])
        else:
            model.RampDown_Constraints.add(model.X[i+1, t-1] - model.X[i+1, t] <= deltaDown[i])

model.Power_Balance_Constraints = ConstraintList()
for t in model.T:
    model.Power_Balance_Constraints.add(sum(model.X[i+1, t] for i in range(N)) >= 1.1 * D[t-1])

model.Run_Constraint = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t], 1))
        else:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t] - model.Run[i+1,t-1] + model.Shut[i+1,t], 1))

ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (True) instead of a Pyomo object. Please modify your rule to return Constraint.Feasible instead of True.

Error thrown for Constraint 'RampDown_Constraints[1]'

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [34]:
from pyomo.environ import *

# Pyomo model
model = ConcreteModel()

# data inputted by human
N = 6        # Number of powerplants
T = 15       # Number of time periods
# Note: All sample data mentioned below is assumed for the purpose of this code
satart = [10324,5678,7802, 12899,4596,9076]   # Sample startup cost data for each powerplant
sdown = [2673, 5893, 982, 6783, 2596, 3561]         # Sample shutdown cost data for each powerplant
fcost = [2000, 3000, 2500, 4000, 3500, 4500]        # Sample fixed cost data for each powerplant
vcost = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data
# Variable cost data (assuming same for each powerplant for all time periods)
lb = [50, 40, 30, 60, 55,65]           # Lower bound of output power (assuming same for each powerplant for all time periods)
ub = [500, 600, 550, 700, 650, 750]           # Upper bound of output power (assuming same for each powerplant for all time periods)
deltaUp = [100, 120, 110, 130, 125, 140]             # Sample data for maximum power increase for each powerplant
deltaDown = [90, 110, 100, 120, 115, 130]            # Sample data for maximum power decrease for each powerplant
D = {1: 1000, 2: 1200, 3: 1300, 4: 1100, 5: 1500, 6: 1400, 7: 1600, 8: 1300, 9: 1700, 10: 1800,
     11: 1900, 12: 1600, 13: 2000, 14: 1800, 15: 1700}  # Sample total power demand for each time
#end
# Sets
model.N = RangeSet(N)
model.T = RangeSet(T)

# Variables
model.X = Var(model.N, model.T, within=NonNegativeReals)
model.Start = Var(model.N, model.T, within=Binary)
model.Shut = Var(model.N, model.T, within=Binary)
model.Run = Var(model.N, model.T, within=Binary)

# Objective
model.obj = Objective(expr=sum(satart[i]*model.Start[i,t] + sdown[i]*model.Shut[i,t] + fcost[i]*model.Run[i,t] +
                           vcost[i,t]*model.X[i, t] for i in range(1,N) for t in model.T), sense=minimize)

# Constraints
model.Capacity_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        model.Capacity_Constraints.add(inequality(lb[i], model.X[i+1, t], ub[i]))

model.RampUp_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampUp_Constraints.add(model.X[i+1, t] <= deltaUp[i])
        else:
            model.RampUp_Constraints.add(model.X[i+1, t] - model.X[i+1, t-1] <= deltaUp[i])

model.RampDown_Constraints = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.RampDown_Constraints.add(- model.X[i+1, t] <= deltaDown[i])
        else:
            model.RampDown_Constraints.add(model.X[i+1, t-1] - model.X[i+1, t] <= deltaDown[i])

model.Power_Balance_Constraints = ConstraintList()
for t in model.T:
    model.Power_Balance_Constraints.add(sum(model.X[i+1,t] for i in range(N)) >= 1.1 * D[t])

model.Run_Constraint = ConstraintList()
for i in range(N):
    for t in model.T:
        if t == 1:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t], 1))
        else:
            model.Run_Constraint.add(inequality(0, model.Run[i+1, t] - model.Start[i+1,t] - model.Run[i+1,t-1] + model.Shut[i+1,t], 1))

solver = SolverFactory('glpk')
solver.solve(model)


{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 555, 'Number of variables': 359, 'Number of nonzeros': 1314, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.04213452339172363}]}